<a href="https://colab.research.google.com/github/Bharathkrishnamurthy/21Day_21_ML-_project/blob/main/DAY12_Image_enhancement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DAY 12:Image Resolution

# Image Super-Resolution using U-Net

This notebook demonstrates how to build and train a U-Net model for image super-resolution. The goal is to take a low-resolution image (64x64) and generate a high-resolution version (128x128).

The U-Net architecture is well-suited for this task as it effectively captures both local and global features through its encoder-decoder structure with skip connections.

### Setup and Imports

This cell imports all the necessary libraries for building and training the U-Net model, including Keras for model definition, OpenCV for image processing, and Matplotlib for visualization.

In [ ]:
!pip install keras

In [ ]:
!pip install tensorflow --upgrade


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 50.5 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.19.0
    Uninstalling tensorboard-2.19.0:
      Successfully uninstalled tensorboard-2.19.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.19.0
    Uninstalling tensorflow-2.19.0:
      Successfully uninstalled tensorflow-2.19.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.12.0 requires tensorflow==2.19.0, but you have tensorflow 2.20.0 which is incompatible.
tensorflow-text 2.19.0 requires tensorflow<2.20,>=2.19.0, but you have tensorflow 2.20.0 which is incompatible.
tf-keras 2.19.0 requires tensorflow<2.20,>=2.19, but you have tensorflow 2.20.0 which is inc

In [ ]:
import os
import cv2 as cv
import numpy as np

import matplotlib.pyplot as plt

from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import (
    Input, BatchNormalization, Activation, Dense, Dropout,
    Conv2D, Conv2DTranspose, MaxPooling2D, GlobalMaxPooling2D, concatenate
)

from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
import tensorflow as tf

### U-Net Model Definition
includes
This cell defines the U-Net model architecture for image super-resolution. The `unet_64to128` function creates a U-Net model that takes a 64x64x3 image as input and outputs a 128x128x3 image. It  an encoder, a bottleneck, and a decoder with skip connections.

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Conv2D, Conv2DTranspose, MaxPooling2D, BatchNormalization,
    Activation, Dropout, concatenate
)

def unet_32to64(input_shape=(32, 32, 3), n_classes=3, final_activation='sigmoid', dropout_rate=0.05):
    inputs = Input(shape=input_shape, name='img')

    # Encoder
    c1 = Conv2D(16, (3,3), padding='same')(inputs)
    c1 = BatchNormalization()(c1); c1 = Activation('relu')(c1)
    c1 = Conv2D(16, (3,3), padding='same')(c1)
    c1 = BatchNormalization()(c1); c1 = Activation('relu')(c1)
    p1 = MaxPooling2D((2,2))(c1); p1 = Dropout(dropout_rate)(p1)   # 32 -> 16

    c2 = Conv2D(32, (3,3), padding='same')(p1)
    c2 = BatchNormalization()(c2); c2 = Activation('relu')(c2)
    c2 = Conv2D(32, (3,3), padding='same')(c2)
    c2 = BatchNormalization()(c2); c2 = Activation('relu')(c2)
    p2 = MaxPooling2D((2,2))(c2); p2 = Dropout(dropout_rate)(p2)   # 16 -> 8

    c3 = Conv2D(64, (3,3), padding='same')(p2)
    c3 = BatchNormalization()(c3); c3 = Activation('relu')(c3)
    c3 = Conv2D(64, (3,3), padding='same')(c3)
    c3 = BatchNormalization()(c3); c3 = Activation('relu')(c3)
    p3 = MaxPooling2D((2,2))(c3); p3 = Dropout(dropout_rate)(p3)   # 8 -> 4

    # Bottleneck (4x4)
    c4 = Conv2D(128, (3,3), padding='same')(p3)
    c4 = BatchNormalization()(c4); c4 = Activation('relu')(c4)
    c4 = Conv2D(128, (3,3), padding='same')(c4)
    c4 = BatchNormalization()(c4); c4 = Activation('relu')(c4)

    # Decoder (upsampling path)
    u5 = Conv2DTranspose(64, (3,3), strides=(2,2), padding='same')(c4)  # 4 -> 8
    u5 = concatenate([u5, c3]); u5 = Dropout(dropout_rate)(u5)
    u5 = Conv2D(64, (3,3), padding='same')(u5)
    u5 = BatchNormalization()(u5); u5 = Activation('relu')(u5)
    u5 = Conv2D(64, (3,3), padding='same')(u5)
    u5 = BatchNormalization()(u5); u5 = Activation('relu')(u5)

    u6 = Conv2DTranspose(32, (3,3), strides=(2,2), padding='same')(u5)  # 8 -> 16
    u6 = concatenate([u6, c2]); u6 = Dropout(dropout_rate)(u6)
    u6 = Conv2D(32, (3,3), padding='same')(u6)
    u6 = BatchNormalization()(u6); u6 = Activation('relu')(u6)
    u6 = Conv2D(32, (3,3), padding='same')(u6)
    u6 = BatchNormalization()(u6); u6 = Activation('relu')(u6)

    u7 = Conv2DTranspose(16, (3,3), strides=(2,2), padding='same')(u6)  # 16 -> 32
    u7 = concatenate([u7, c1]); u7 = Dropout(dropout_rate)(u7)
    u7 = Conv2D(16, (3,3), padding='same')(u7)
    u7 = BatchNormalization()(u7); u7 = Activation('relu')(u7)
    u7 = Conv2D(16, (3,3), padding='same')(u7)
    u7 = BatchNormalization()(u7); u7 = Activation('relu')(u7)

    # Final upsample to reach 64x64
    u8 = Conv2DTranspose(16, (3,3), strides=(2,2), padding='same')(u7)   # 32 -> 64
    u8 = Dropout(dropout_rate)(u8)
    u8 = Conv2D(16, (3,3), padding='same')(u8)
    u8 = BatchNormalization()(u8); u8 = Activation('relu')(u8)

    outputs = Conv2D(n_classes, (1,1), activation=final_activation, name='output')(u8)

    return Model(inputs=inputs, outputs=outputs, name='UNet_32to64')

# Build and print summary
model = unet_32to64()
model.summary()


Model: "UNet_32to64"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ img (InputLayer)    │ (None, 32, 32, 3) │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 32, 32,    │        448 │ img[0][0]         │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 32, 32,    │         64 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 32, 32,    │          0 │ batch_normalizat… │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 32, 32,    │      2,320 │ activation[0][0]  │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │         64 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 32, 32,    │          0 │ batch_normalizat… │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 16, 16,    │          0 │ activation_1[0][… │
│ (MaxPooling2D)      │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 16, 16,    │          0 │ max_pooling2d[0]… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 16, 16,    │      4,640 │ dropout[0][0]     │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16, 16,    │        128 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 16, 16,    │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 16, 16,    │      9,248 │ activation_2[0][… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16, 16,    │        128 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 16, 16,    │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 8, 8, 32)  │          0 │ activation_3[0][… │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 8, 8, 32)  │          0 │ max_pooling2d_1[… │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 543,347 (2.07 MB)

 Trainable params: 541,907 (2.07 MB)

 Non-trainable params: 1,440 (5.62 KB)

### Model Initialization and Compilation

This cell initializes the U-Net model with the specified input shape and number of classes. It then compiles the model using the Adam optimizer and mean absolute error (MAE) as the loss function. The input and output shapes are printed to verify the model architecture.

In [ ]:
model = unet_32to64(input_shape=(32,32,3), n_classes=3, final_activation='sigmoid')

print('Input shape:', model.input_shape)
print('Output shape:', model.output_shape)
model.compile(optimizer=Adam(1e-4), loss='mae' )

Input shape: (None, 32, 32, 3)
Output shape: (None, 64, 64, 3)


In [ ]:
!ls /kaggle/input
!ls /kaggle/input/celeba-dataset



ls: cannot access '/kaggle/input/celeba-dataset': No such file or directory


### Data Generator

This cell defines a data generator function `datagen` that loads and preprocesses images from the CelebA dataset. It resizes the images to 128x128 as ground truth and to 64x64 as low-resolution input, and normalizes the pixel values. The generator yields batches of low-resolution and high-resolution image pairs for training.

In [ ]:
import tensorflow as tf
import numpy as np

(x_train, _), (_, _) = tf.keras.datasets.cifar10.load_data()
x_train = x_train.astype('float32') / 255.0

def datagen(batch_size):
    while True:
        idx = np.random.randint(0, len(x_train), batch_size)
        x = x_train[idx]
        y = tf.image.resize(x, (128,128)).numpy()
        yield x, y


In [ ]:
# ---- datagen that yields 32x32 -> 64x64 ----
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10

(x_train, _), (_, _) = cifar10.load_data()
x_train = x_train.astype('float32') / 255.0  # shape (50000,32,32,3)

def datagen_cifar(batch_size):
    n = x_train.shape[0]
    while True:
        idx = np.random.randint(0, n, size=batch_size)
        x_batch = x_train[idx]                      # already 32x32
        # create 64x64 targets by upsampling
        y_batch = tf.image.resize(x_batch, (64,64)).numpy()
        yield x_batch, y_batch

# ---- build model (32 -> 64) ----
model = unet_32to64(input_shape=(32,32,3), n_classes=3, final_activation='sigmoid')

model.compile(optimizer='adam', loss='mae', metrics=['mse'])
model.summary()

# ---- train ----
batch_size = 32
num_images = 10           # pretend we have 10k samples per epoch
steps_per_epoch = num_images // batch_size

results = model.fit(
    datagen_cifar(batch_size),
    steps_per_epoch=steps_per_epoch,
    epochs=3,
    verbose=1
)


Model: "UNet_32to64"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ img (InputLayer)    │ (None, 32, 32, 3) │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_45 (Conv2D)  │ (None, 32, 32,    │        448 │ img[0][0]         │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │         64 │ conv2d_45[0][0]   │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_45       │ (None, 32, 32,    │          0 │ batch_normalizat… │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_46 (Conv2D)  │ (None, 32, 32,    │      2,320 │ activation_45[0]… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │         64 │ conv2d_46[0][0]   │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_46       │ (None, 32, 32,    │          0 │ batch_normalizat… │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_9     │ (None, 16, 16,    │          0 │ activation_46[0]… │
│ (MaxPooling2D)      │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_21          │ (None, 16, 16,    │          0 │ max_pooling2d_9[… │
│ (Dropout)           │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_47 (Conv2D)  │ (None, 16, 16,    │      4,640 │ dropout_21[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16, 16,    │        128 │ conv2d_47[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_47       │ (None, 16, 16,    │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_48 (Conv2D)  │ (None, 16, 16,    │      9,248 │ activation_47[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16, 16,    │        128 │ conv2d_48[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_48       │ (None, 16, 16,    │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_10    │ (None, 8, 8, 32)  │          0 │ activation_48[0]… │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_22          │ (None, 8, 8, 32)  │          0 │ max_pooling2d_10… │
│ (Dropout)           │                   │            │                 

 Total params: 543,347 (2.07 MB)

 Trainable params: 541,907 (2.07 MB)

 Non-trainable params: 1,440 (5.62 KB)

Epoch 1/3
  16774/Unknown 10809s 644ms/step - loss: 0.0260 - mse: 0.0015

In [ ]:
# Simple end-to-end: 32x32 -> 64x64 using CIFAR-10 (no external files)
import os, numpy as np, matplotlib.pyplot as plt, cv2 as cv
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, MaxPooling2D, BatchNormalization, Activation, Dropout, concatenate

# ---- tiny UNet (fast & simple) ----
def tiny_unet_32to64(input_shape=(32,32,3)):
    inp = Input(input_shape)
    # encoder
    c1 = Conv2D(32,3,padding='same',activation='relu')(inp)
    p1 = MaxPooling2D(2)(c1)                     # 32 -> 16
    c2 = Conv2D(64,3,padding='same',activation='relu')(p1)
    p2 = MaxPooling2D(2)(c2)                     # 16 -> 8
    # bottleneck
    b = Conv2D(128,3,padding='same',activation='relu')(p2)
    # decoder
    u1 = Conv2DTranspose(64,3,strides=2,padding='same',activation='relu')(b)  # 8 -> 16
    u1 = concatenate([u1, c2])
    u2 = Conv2DTranspose(32,3,strides=2,padding='same',activation='relu')(u1)  # 16 -> 32
    u2 = concatenate([u2, c1])
    # final upsample to 64
    up = Conv2DTranspose(32,3,strides=2,padding='same',activation='relu')(u2)  # 32 -> 64
    out = Conv2D(3,1,activation='sigmoid')(up)
    return Model(inp, out)

# ---- data: CIFAR-10 (32x32) -> targets 64x64 ----
(x_train, _), (_, _) = tf.keras.datasets.cifar10.load_data()
x_train = x_train.astype('float32') / 255.0

def datagen(batch_size):
    n = x_train.shape[0]
    while True:
        idx = np.random.randint(0, n, batch_size)
        lr = x_train[idx]                                  # 32x32
        hr = tf.image.resize(lr, (64,64)).numpy()         # 64x64 targets
        yield lr, hr

# ---- helper to show/save samples ----
def show_save(model, epoch, save_dir='samples'):
    os.makedirs(save_dir, exist_ok=True)
    idx = np.random.choice(len(x_train), 4, replace=False)
    figs = []
    for i,j in enumerate(idx):
        lr = x_train[j]
        hr = tf.image.resize(lr, (64,64)).numpy()
        pred = model.predict(np.expand_dims(lr,0), verbose=0)[0]
        # upscale for display (128x128)
        orig_up = cv.resize((lr*255).astype('uint8'), (128,128))
        hr_up   = cv.resize((hr*255).astype('uint8'), (128,128))
        lr_up   = cv.resize((cv.resize((lr*255).astype('uint8'), (64,64))), (128,128))
        pred_up = cv.resize((np.clip(pred,0,1)*255).astype('uint8'), (128,128))
        row = np.hstack([orig_up, hr_up, lr_up, pred_up])
        figs.append(row)
    grid = np.vstack(figs)
    plt.figure(figsize=(8, 8))
    plt.imshow(grid); plt.axis('off')
    path = os.path.join(save_dir, f'epoch_{epoch+1:02d}.png')
    plt.savefig(path, bbox_inches='tight'); plt.show()
    print('Saved:', path)

# ---- build, compile, train ----
model = tiny_unet_32to64()
model.compile(optimizer='adam', loss='mae', metrics=['mse'])
model.summary()

batch_size = 32
steps_per_epoch = 100   # small for demo; increase if you want longer epochs
epochs = 3

# callback to show/save at epoch end
class ShowSaveCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        show_save(self.model, epoch)

model.fit(
    datagen(batch_size),
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    callbacks=[ShowSaveCallback()],
    verbose=1
)
